In [9]:
from dotenv import load_dotenv
import os

load_dotenv(verbose=True)
key = os.getenv('OPENAI_API_KEY')

In [10]:
from bs4 import BeautifulSoup
from langchain.agents import tool
import requests
import re

In [11]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

In [12]:
from langchain.agents import create_tool_calling_agent
from langchain.agents import AgentExecutor

In [13]:
@tool
def get_word_length(word: str) -> int:          # 단어의 길이를 리턴하는 도구
    """Returns the length of a word."""

    return len(word)

In [14]:
@tool
def add_function(a: float, b: float) -> float:  # 두 수의 합을 리턴하는 도구
    """Adds two numbers together."""
    
    return a + b

In [15]:
@tool
def naver_news_crawl(news_url: str) -> str:                 # 네이버 뉴스기사를 크롤링하여 본문 내용을 리턴하는 도구
    """Crawls a 네이버(naver.com) news article and returns the body content."""

    response = requests.get(news_url)                       # HTTP GET 요청

    if response.status_code == 200:                         # 응답 200. 성공
        soup = BeautifulSoup(response.text, 'html.parser')

        title = soup.find('h2', id='title_area').get_text()
        content = soup.find('div', id='contents').get_text()    
        
        cleaned_title = re.sub(r'\n{2, }', '\n', title)
        cleaned_content = re.sub(r'\n{2, }', '\n', content)
    else:
        print(f'HTTP 요청 실패. {response.status_code}')
    
    return f'{cleaned_title}\n{cleaned_content}'

In [16]:
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system", 
            "You are very powerful assistant, but don't know current events"
        ),
        ("user", "{input}"),                                        # 사용자 입력
        
        # 에이전트가 검색하는 과정들이나 내용 등을 끄적이는 메모장 같은 공간을 플레이스 홀더로 만들어준다
        MessagesPlaceholder(variable_name="agent_scratchpad")
    ]
)

In [17]:
llm = ChatOpenAI(
    api_key=key, 
    model='gpt-4o-mini', 
    temperature=0
)

In [18]:
tools = [get_word_length, add_function, naver_news_crawl]   # 여러 개의 도구를 한꺼번에 바인딩 위해서 리스트에 넣기

In [19]:
agent = create_tool_calling_agent(llm, tools, prompt)       # 에이전트       

In [20]:
agent_executor = AgentExecutor(
    agent=agent, 
    tools=tools, 
    verbose=True,
    handle_parsing_errors=True
)

In [21]:
answer = agent_executor.invoke({'input': '랭체인 이라는 단어의 글자 길이는?'})



> Entering new AgentExecutor chain...

Invoking: `get_word_length` with `{'word': '랭체인'}`


3"랭체인"이라는 단어의 글자 길이는 3입니다.

> Finished chain.


In [22]:
print(answer['output'])

"랭체인"이라는 단어의 글자 길이는 3입니다.


In [23]:
answer = agent_executor.invoke({'input': '90 + 10 = ?'})



> Entering new AgentExecutor chain...

Invoking: `add_function` with `{'a': 90, 'b': 10}`


100.090 + 10 = 100.

> Finished chain.


In [24]:
print(answer['output'])

90 + 10 = 100.
